In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import supy as sp
from pathlib import Path
import json

/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
print('Initializing SUEWS variables.....')
df_state_init, df_forcing = sp.load_SampleData()

path_runcontrol = Path('./run') / 'RunControl.nml'
df_state_init = sp.init_supy(path_runcontrol)

Initializing SUEWS variables.....


In [3]:
# vars_dict={}
# for var in df_state_init.columns.levels[0]:
#     vars_dict[var]={'logic':'range','param':{'min':0,'max':10}}
    
# with open('check_file_suews_general.json', 'w') as fp:
#     json.dump(vars_dict, fp)

In [4]:
def open_check_file(check_file):
    with open(check_file) as cf:     
        cr = json.load(cf)
    return cr
#---------------------------------------------
def is_accepted_fun(is_accepted_flag):
    
    if(not is_accepted_flag):
        return 'No'
        
    else:
        return 'Yes'
#---------------------------------------------
def check_range(var, values, cr):

    min_v = cr[var]['param']['min']
    max_v = cr[var]['param']['max']
    
    if min_v == '-inf':
        min_v = -np.inf
    if max_v == 'inf':
        max_v = np.inf
        
    description = 'Value = '+str(values)
    suggestion = 'should be between '+str(min_v)+' and '+str(max_v)
    is_accepted_flag = False

    for value in np.nditer(values):
        if (min_v <= value <= max_v) or value == -999:
            is_accepted_flag = True

    is_accepted = is_accepted_fun(is_accepted_flag)
        
    return [var, is_accepted, description, suggestion]
#---------------------------------------------
def check_method(var, values, cr):
    allowed_range = cr[var]['param']['allowed']

        
    description = 'Value = '+str(values)
    suggestion = 'should be one of '+str(allowed_range)
    is_accepted_flag = False

    for value in np.nditer(values):
        if (value in allowed_range) or value == -999:
            is_accepted_flag = True
    is_accepted = is_accepted_fun(is_accepted_flag)
    
    return [var, is_accepted, description, suggestion]
#---------------------------------------------
def check_NA(var, values, cr):
    description = 'Value = '+str(values)
    return [var, 'NA', description, '']
#---------------------------------------------
def check_var_suews(var, values, cr, df_sum):
    logic = cr[var]['logic']
    if logic == 'range':
        out_list = check_range(var, values, cr)
    elif logic == 'method':
        out_list = check_method(var, values, cr)
    elif logic == 'NA':
        out_list = check_NA(var, values, cr)
        
    df_sum.loc[len(df_sum)] = out_list
    return df_sum



In [5]:
check_file = 'check_file_suews_general.json'
cr = open_check_file(check_file)
df_sum = pd.DataFrame(columns=['Parameter', 'Is acceptable?', 'Description', 'Suggestion'])
for var in df_state_init.columns.levels[0]: 
        df_sum = check_var_suews(var, df_state_init[var].iloc[0].values, cr, df_sum)
df_sum.to_csv('checker_log.csv')
df_sum

,Parameter,Is acceptable?,Description,Suggestion
0,ah_min,Yes,Value = [15. 15.],should be between 0 and inf
1,ah_slope_cooling,Yes,Value = [2.7 2.7],should be between 0 and inf
2,ah_slope_heating,Yes,Value = [2.7 2.7],should be between 0 and inf
3,ahprof_24hr,Yes,Value = [0.57 0.65 0.45 0.49 0.43 0.46 0.4 0....,should be between 0 and inf
4,alb,Yes,Value = [0.12 0.15 0.12 0.18 0.21 0.21 0.1 ],should be between 0 and 1
5,albmax_dectr,Yes,Value = [0.18],should be between 0 and 1
6,albmax_evetr,Yes,Value = [0.12],should be between 0 and 1
7,albmax_grass,Yes,Value = [0.21],should be between 0 and 1
8,albmin_dectr,Yes,Value = [0.12],should be between 0 and 1
9,albmin_evetr,Yes,Value = [0.11],should be between 0 and 1


In [13]:
df_sum[df_sum['Is acceptable?']=='No']

,Parameter,Is acceptable?,Description,Suggestion
